## Tutorial: Basics of MIP Modelling

In this tutorial session, we will learn how to use the FICO Xpress and SCIP Python interfaces to efficiently model and solve basic mixed-integer programming problems via a series of in-class exercises.

Instructions:
- the tutorial is hands-on, with text descriptions, mathematical formulations, and code snippets shown along the notebook to be analyzed, run and understood in the order they appear.
- it is divided in 4 parts, which will be solved using the FICO Xpress Python interface. Parts 1S and 2S, at the end of the notebook, correspond to the first two parts but using the SCIP Python interface.
  - you are expected to solve, in class, all the assignments marked as **EXERCISE**.

Software requirements:
- if running in cloud (recommended):
  - https://github.com/features/codespaces (recommended)
  - https://colab.research.google.com/
  - (no Python installation needed, but you need to install "xpress", "PySCIPOpt" packages, and other packages in the remote machine by running "!pip install [module]" - see first code cell)
- if run locally (if you are an expert user):
  - your favorite IDE that supports Interactive Python Notebooks, such as VS Code, PyCharm, or Jupyter notebook
  - Python installation >= 3.9
  - Xpress package >= 9.4
  - PySCIPOpt package >= 5.1

Examples and documentation:
* Check the [Xpress Python API reference manual](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML) and the [Xpress Python examples](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chExamples.html) online, and find a PDF with training slides named "xpress_python_api.pdf" in the shared GitHub repo.
* SCIP Python interface: https://github.com/scipopt/PySCIPOpt

Licensing guidelines:
- FICO Xpress Optimizer requires a full license for some of the exercises. You will receive a license file via e-mail, which you should upload into the root directory of your codespace.
  - an environment variable named XPAUTH_PATH must be set to (the path to) the license file "xpauth.xpr" (automatically done in Codespaces, see code cell below for Google Colab).
  - <span style="color:red">the provided license must only be used for the purposes of the co@work summer school</span>. Please join the [FICO Academic Partner Program](https://community.fico.com/s/academic-programs) to access full licenses of Xpress free of charge.


In [6]:
# Install Python packages (if running in cloud)
!pip install xpress PySCIPopt        # NumPy is installed along with xpress

# If working in Google Colab, uncomment the following lines to set an environment XPAUTH_PATH variable to the license file (assumes the license file is in the same working directory as the notebook)
# import os
# os.environ['XPAUTH_PATH'] = 'xpauth.xpr'

In [5]:
# To disable warnings regarding the Xpress license, run this code cell
import xpress as xp
import warnings
warnings.simplefilter('ignore', xp.LicenseWarning)

### Part 1 - Creating and solving a basic LP (Xpress)

Let's start by creating and solving a simple linear program:

$$
\begin{array}{lll}
  \min & x + y\\
  \textrm{s.t.} & 2x + 3y \ge 6\\
                & 4x + 2y \ge 7\\
                & x,y \ge 0
\end{array}
$$

Analyze and run the code below to become familiar with problem creation, adding variables, contraints and objectives, and solving a problem using the FICO Xpress interface:

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

# First step is to create a new problem 
p = xp.problem()

# Create and add decision variables directly (always first to be added)
x = p.addVariable(name='myvar_1') # variables are continuous and non-negative by default
y = p.addVariable(name='myvar_2') # use the vartype argument to define other variable types such as 'vartype=xp.integer' or 'vartype=xp.binary'

# The define and add constraints and objective (in no particular order)
constr1 = 2*x + 3*y >= 6 # can be constructed using operator overloading
p.addConstraint(constr1, 4*x + 2*y >= 7) # constraints can be added by passing an object, a constraint expression, or lists and dictionaries thereof

obj = x + y
p.setObjective(obj)  # minimization by default

# Uncomment the following line if you want to turn off logging from Xpress:
# p.controls.outputlog = 0

p.optimize() # trigger the Xpress optimizer to solve the problem

Now let's query the optimal objective value, and print primal and dual solution values using `problem.getSolution` for the primal solution and `problem.getDual` for the dual solution (dual solutions can only be obtained for linear optimization models). 

Note that these functions can be called without arguments (to obtain the whole vector), or with a variable/constraint object to get one or more values.

In [ ]:
# FICO Xpress Optimizer - Python Interface

print(f'Objective value={p.getObjVal()}')

print(f'Solution: x={p.getSolution(x)}, y={p.getSolution(y)}')
# print(f'solution: x={p.getSolution('myvar_1')}, y={p.getSolution('myvar_2')}') # Same effect with this call that uses variable names instead of objects

print(f'Dual: constr1->{p.getDual(constr1)}')

**EXERCISE 1**: Add the constraint $x + 2y \le 2$ to the problem, then optimize the problem again. The problem should now be infeasible.

Bonus: after `p.optimize()`, call the [problem.iisall](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.iisall.html) function of the Xpress Python interface to verify which constraints and/or variable bounds made the problem infeasible.

In [ ]:
# FICO Xpress Python interface (solution)



### Part 2 - Modeling and solving a Knapsack problem using NumPy (Xpress)

Now let's move on to Mixed-Integer Programming: let's formulate and solve a [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem) with the following value/weight vectors:

$$
\begin{array}{lllrrrrrrr}
v& =& (12,&15,& 9,&11,& 8,& 7,&5)\\
w& =& (13, &18,& 9,& 12,& 8,& 10,& 4)
\end{array}
$$

and with knapsack capacity $C=40$. The formulation of a knapsack problem is as follows:

$$
\begin{array}{lllrrrrrrr}
\max & \sum_{i=1}^n v_i x_i\\
\textrm{s.t.} & \sum_{i=1}^n w_i x_i \le C\\
& x_i \in \{0,1\}, \forall i=1,\ldots{},n
\end{array}
$$

The problem can be modeled as follows using the [xpress.Sum](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Sum.html) operator for modeling constraints and the objective function:

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

v = [12,15,9,11,8, 7,5]
w = [13,18,9,12,8,10,4]
C = 40

p = xp.problem()

n = len(v)
x = [p.addVariable(vartype=xp.binary) for _ in range(n)]  # create a binary variable for each item and store in a list

k_con = xp.Sum(w[i]*x[i] for i in range(n)) <= C # method xp.Sum accepts lists containing Xpress objects as argument(s)
p.addConstraint(k_con)

k_obj = xp.Sum(v[i]*x[i] for i in range(n))
p.setObjective(k_obj, sense=xp.maximize) # optimization sense is min by default

p.optimize()

print(p.getSolution())

#### Using NumPy arrays - an example

**NumPy** is an essential toolbox for Python users. The Xpress Python interface can handle (multi-)arrays of floats, variables, expressions, constraints as naturally as with lists thereof and leverage the broadcasting features of NumPy arrays. Models [using NumPy arrays](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chNumpy.html) are handled more efficiently by the Xpress Python interface, making the model building time significantly lower for large-scale  problems with a high number of variables and/or constraints.

The example below creates a random 20x30 NumPy matrix $A$ and vectors ${\bf b}$ and ${\bf c}$, then creates a problem with variables ${\bf x}$ added as a NumPy array by using [p.addVariables](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/problem.addVariables.html), ${\bf c}$ as objective function coefficient vector and constraints $A{\bf x} \le {\bf b}$ using the [xpress.Dot](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/xpress.Dot.html) operator:

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp
import numpy as np

n = 30 # number of variables
m = 20 # number of constraints

A = np.random.random((m,n))
b = np.random.random(m)
c = np.random.random(n)

p = xp.problem()

x = p.addVariables(n) # when p.addVariables receives integer arguments, np arrays are created

# xp.Dot only works with np arrays (of numbers and/or Xpress objects: expressions and variables)
consys = xp.Dot(A, x) <= b
p.addConstraint(consys)

objective= xp.Dot(c, x)
p.setObjective(objective, sense=xp.maximize) # optimization sense is min by default

p.optimize()

#### Modeling the knapsack problem using NumPy arrays

**EXERCISE 2**: Rewrite the knapsack problem using NumPy constructs and the xp.Dot operator in the code cell below, solve it and print the optimal solution vector

In [ ]:
# FICO Xpress Optimizer - Python Interface (solution)



### Coffee break

### Part 3 - Reformulating a quadratic problem (MIQP) as a MIP (Xpress)

We'll implement a simple reformulation of a Binary Box-constrained Quadratic Programming problem:

$$
\begin{array}{ll}
\min & x^T Q x + c^T x\\
\textrm{s.t.} & x \in \{0,1\}^n
\end{array}
$$

This problem has a quadratic objective function and all variables are binary; furthermore, the only constraints are the implicit bounds on binary variables, i.e. $\{0,1\}$. The problem can be written in algebraic form as follows:

$$
\begin{array}{ll}
\min &\sum_{i=1}^n\sum_{j=1}^n q_{ij} x_i x_j + \sum_i c_i x_i\\
\textrm{s.t.} & x_i\in \{0,1\} & \forall i=1,2,\ldots, n.
\end{array}
$$

where $Q$ is a symmetric matrix. Let's try an example:

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - .5
c = np.random.random(n) - .5

p = xp.problem()

x = p.addVariables(n, vartype=xp.binary)

p.setObjective(xp.Dot(x,Q,x) + xp.Dot(c,x))

p.optimize()

The problem can be solved as a MIP after a *reformulation* that does the following:

* Creates a (binary) variable $y_{ij}$ for each product $x_i x_j$;
* For each variable $y_{ij}$, it adds the following constraints linking $y$ with $x$ variables:

$$
\begin{array}{lll}
y_{ij} &\le& x_i \\
y_{ij} &\le& x_j \\
y_{ij} &\ge& x_i + x_j - 1
\end{array}
$$

After this reformulation, the problem becomes a MIP, as follows:

$$
\begin{array}{ll}
\min & \sum_{i=1}^n \sum_{j=1}^n q_{ij} y_{ij} + \sum_{i=1}^n c_i x_i\\
\textrm{s.t.} &y_{ij} \le x_i  &\forall i,j\\
&y_{ij} \le x_j  &\forall i,j\\
&y_{ij} \ge x_i + x_j - 1 &\forall i,j\\
&x_i \in \{0,1\} \forall i=1,2,\ldots,n\\
&y_{ij} \in \{0,1\} \forall i,j=1,2,\ldots,n\\
\end{array}
$$

**EXERCISE 3**: implement the reformulation above explicitly by introducing the variables $y_{ij}$, the new contraints and a modified objective function in the TO DO section below

In [ ]:
# FICO Xpress Optimizer - Python Interface (solution)

import xpress as xp

np.random.seed(1234567)

n = 30
Q = np.random.random((n,n)) - 0.5
c = np.random.random(n) - 0.5

p = xp.problem()

x = p.addVariables(n, vartype=xp.binary)

# TO DO - add dec. vars, constraints and objective function


p.optimize()

### Part 4 - Modeling and solving an electricity generation problem (Xpress)

In this part, we will model and solve an electricity generation problem from [Garver (1963)](https://ieeexplore.ieee.org/document/4501405) found in real power markets:

Four types of power generators are available to meet daily electricity demand and a security reserve of up to 20% above. Each type of generator has a set minimum and maximum power output.
A generator can only be started or stopped at the beginning of a time period. The objective is to determine which generators should be used, and at which power level, in each period so that total daily cost is minimized.

Three arrays of binary variables are required to determine when to 'start' and 'stop' generators, and decide which ones are set to 'work' in each time period. Another set of real variables are introduced to represent the additonal energy production ('padd') of each working unit type above its minimum output level. Variable 'start' is the difference between 'work' in a time period and 'work' in the preceding period, and 'stop' is the difference between the previous time period and the current one.

This unit commitment problem can be mathematically formulated as follows:
$$
\begin{align*}
\min & \quad \sum_{u \in \mathcal{U}} \sum_{t \in \mathcal{T}} C_{S_u}^{\rm start} \cdot start_{u,t} + L_t \cdot (C_{S_u}^{\rm min} \cdot work_{u,t} + C_{S_u}^{\rm add} \cdot padd_{u,t}) 
+ \sum_{u \in \mathcal{U}} \sum_{t \in \mathcal{T}} stop_{u,t} \cdot PEN \\

\hbox{subject to:} \\
& \hbox{If generator starts in period:} \\
& \qquad start_{u,t}  \geq work_{u,t}  - work_{u,n} , \qquad \forall u \in U, \forall t \in \mathcal{T}, n = (NT+t-1) \hbox{ \% } NT \\
& \qquad start_{u,t}  \leq work_{u,t}  , \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T} \\
& \hbox{If generator stops in period:} \\
& \qquad stop_{u,t}  \geq work_{u,n}  - work_{u,t} , \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T}, n = (NT+t-1) \hbox{ \% } NT \\
& \qquad stop_{u,t}  \leq 1 - work_{u,t}, \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T} \\
& \hbox{Limit on power production above minimum level:} \\
& \qquad padd_{u,t}\leq (P_{S_u}^{\rm max}-P_{S_u}^{\rm min}) \cdot work_{u,t}, \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T} \\
& \hbox{Satisfy daily electricity demand:} \\
& \qquad \sum_{u \in \mathcal{U}} P_{S_u}^{\rm min} \cdot work_{u,t} +  padd_{u,t} \geq D_t, \qquad \forall t \in \mathcal{T} \\
& \hbox{Ensure security reserve of 20\%:} \\
& \qquad \sum_{u \in \mathcal{U}} P_{S_u}^{\rm max} \cdot work_{u,t} \geq 1.20 \cdot D_t, \qquad \forall t \in \mathcal{T} \\
\end{align*}
$$

Where:
* $\mathcal{T}$ = set of time periods
* $NT$ = number of time periods ($NT = |\mathcal{T}|$)
* $\mathcal{S}$ = set of generator types
* $A_s$ = available number of type $s$ generators
* $\mathcal{U}$ = set of unit generators to be committed ($|\mathcal{U}| = \sum_{s \in \mathcal{S}} A_s$)
* $L_t$ = length of time period $t$
* $D_t$ = demand in time period $t$
* $S_u$ = type of generator $u$
* $C_{s}^{\rm start}$ = start-up cost of generator type $s$
* $C_{s}^{\rm min}$ = hourly cost of operating generator $s$ at minimum output
* $C_{s}^{\rm add}$ = cost/hour/MW of prod. above min. level
* $P_{s}^{\rm min}$,$P_{s}^{\rm max}$ = minimum and maximum output of a generator type $s$
* $PEN$ = penalty to ensure that stop variables are 0 when not needed

The decision variables are:
* $start_{u,t}$ = 1 if generator $u$ is started in period $t$, 0 otherwise
* $stop_{u,t}$ = 1 if generator $u$ is stopped in period $t$, 0 otherwise
* $work_{u,t}$ = 1 if generator $u$ is working during period $t$, 0 otherwise
* $padd_{u,t}$ = production level (MW) of generator $u$ above the minimum level $P_{S_u}^{\rm min}$ in period $t$

**EXERCISE 4.1**: Analyze the problem description and formulation, and fill in the five "TO DO" sections below to complete the modelling of the above formulation and print the results using the Xpress Python interface.

In [ ]:
# FICO Xpress Optimizer - Python Interface

import xpress as xp

# Time periods
LEN = [6, 3, 3, 2, 4, 4, 2]
DEM = [12000, 32000, 25000, 36000, 25000, 30000, 18000]

# Power plants
PMIN = [750, 1000, 1200, 1800]      # minimum output (MW) per generator type
PMAX = [1750, 1500, 2000, 3500]     # maximum output (MW) per generator type
CSTART = [5000, 1600, 2400, 1200]   # start-up cost () per generator type
CMIN = [2250, 1800, 3750, 4800]     # hourly cost of operating generator type at minimum output (see PMIN)
CADD = [2.7, 2.2, 1.8, 3.8]         # cost/hour/MW of prod. above min. level per generator type
AVAIL = [10, 4, 8, 3]               # number of units per type

NT = 7
PERIODS = range(NT)         # Time periods
TYPES = range(4)            # Power generator types
UNITS = range(sum(AVAIL))   # Power generation units
MAPU = [i for i in TYPES for p in range(AVAIL[i])]      # Associating units with types

# Create problem
p = xp.problem("Unit commitment")

# Create decision variables
start = p.addVariables(UNITS, PERIODS, vartype=xp.binary, name='start')
stop = p.addVariables(UNITS, PERIODS, vartype=xp.binary, name='stop')
work = p.addVariables(UNITS, PERIODS, vartype=xp.binary, name='work')
padd = p.addVariables(UNITS, PERIODS, name='padd')

# If generator starts in period
p.addConstraint(start[u,t] >= work[u,t] - work[u,(NT+t-1) % NT] for u in UNITS for t in PERIODS)
p.addConstraint(start[u,t] <= work[u,t]  for u in UNITS for t in PERIODS)

# If generator stop before period
p.addConstraint(stop[u,t] >= work[u,(NT+t-1) % NT] - work[u,t] for u in UNITS for t in PERIODS)
p.addConstraint(stop[u,t] <= 1 - work[u,t] for u in UNITS for t in PERIODS)

# **TO DO** Limit on power production above minimum level

# **TO DO** Satisfy demands

# **TO DO** Security reserve of 20%

# **TO DO** Create and add the objective function of the problem (hint: compute 'daily cost' and 'penalty' separately)

# **TO DO** Optimize the problem and print the daily cost, penalty and total objective value

Now let's print the solution in a user-friendly way:

In [ ]:
# FICO Xpress Optimizer - Python Interface

# Display solution
def print_sol(prob):
    ct = 0
    print(f"Time period          ", end="")
    for t in PERIODS:
        print('{:5d}-{:2}'.format(ct,ct+LEN[t]), end="")
        ct += LEN[t]

    for u in UNITS:
        print("\n",f"\nUnit {u+1} Working      ", end="")
        for t in PERIODS:
            print("     off" if prob.getSolution(work[u,t]) == 0 else "      on", end="")
        print("\n       Status change", end="")
        for t in PERIODS:
            if prob.getSolution(stop[u,t]) > 0.5: print("    stop", end="")
            else:
                if prob.getSolution(start[u,t]) > 0.5: print("    start", end="")
                else: print("       -", end="")
        print("\n       Total output ", end="")
        for t in PERIODS:
            print('{:8d}'.format(round(prob.getSolution(PMIN[MAPU[u]]*work[u,t] + padd[u,t]))), end="")
        print("\n       of which add.", end="")
        for t in PERIODS:
            print('{:8d}'.format(round(prob.getSolution(padd[u,t]))), end="")

print_sol(p)

##### Bonus part: Adding indicator constraints

In real-world applications, generators often must remain in a certain state after they have been switched to that state, i.e. if a generator is turned ON/OFF, it must remain in that state for at least $ON_s^{\rm min}$/$OFF_s^{\rm min}$ periods, respectively. This type of constraints are known as **indicator constraints** as their application depends on the value of a binary variable, called the 'indicator'. 

The indicator constraints can be formulated as follows:

$$
\begin{align*}
& \hbox{Can only switch OFF at least $ON_s^{\rm min}$ periods after it has been turned ON:} \\
& \qquad \sum_t^{t+ON_s^{\rm min}} stop_{u,n}  \leq 0, \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T}: start_{u,t} = 1, n = t \hbox{ \% } NT \\
& \hbox{Can only switch ON at least $OFF_s^{\rm min}$ periods after it has been turned OFF:} \\
& \qquad \sum_t^{t+OFF_s^{\rm min}} start_{u,n}  \leq 0, \qquad \forall u \in \mathcal{U}, \forall t \in \mathcal{T}: stop_{u,t} = 1, n = t \hbox{ \% } NT \\
\end{align*}
$$

where $ON_s^{\rm min}$,$OFF_s^{\rm min}$ = minimum time intervals a generator type $s$ must be ON/OFF once it has switched to that state, respectively.

**Bonus Exercise 4.2**: Add the indicator constraints to the model in the code cell below, run the cell and compare the outcomes. What is the extra cost introduced by the loss of flexibility ? Also confirm that, contrary to the previous run, units are not allowed to be turned ON and then OFF within 3 time periods by printing the solution.

Hint: use the [problem.addIndicator()](https://www.fico.com/fico-xpress-optimization/docs/latest/solver/optimizer/python/HTML/chModeling.html?scroll=secModelingIndicator) method to add the indicator constraints.

In [ ]:
# FICO Xpress Optimizer - Python Interface

# Minimum time 
ONMIN = [3, 3, 3, 3]                # minimum time intervals a generator type $s$ must be ON once switched to that state
DWMIN = [3, 3, 3, 3]                # minimum time intervals a generator type $s$ must be OFF once switched to that state

# **TO DO** Insert the indicator constraints here

# **TO DO** Optimize the problem and print the daily cost, penalty and total objective value

print_sol(p)

### Part 1S - Creating and solving a basic LP (SCIP)

Let's start by creating and solving a simple linear program:

$$
\begin{array}{lll}
  \min & x + y\\
  \textrm{s.t.} & 2x + 3y \ge 6\\
                & 4x + 2y \ge 7\\
                & x,y \ge 0
\end{array}
$$

Analyze and run the code below to become familiar with problem creation, adding variables, contraints and objectives, and solving a problem using the SCIP interface:

In [ ]:
# SCIP - Python Interface

from pyscipopt import Model

# First step is to create a new problem 
model = Model()

# Create and add decision variables directly (always first to be added)
x = model.addVar(name='myvar_1') # variables are continuous and non-negative by default
y = model.addVar(name='myvar_2') # use the vartype argument to define other variable types such as 'vtype="Integer"' or 'vtype="Binary"'

# Then constraints and objective (in no order)
constr1 = model.addCons(2*x + 3*y >= 6) # can be constructed using operator overloading
constr2 = model.addCons(4*x + 2*y >= 7)

obj = x + y
model.setObjective(obj)  # minimization by default

# Uncomment the following line if you want to turn off logging from PySCIPOpt:
model.hideOutput(False)

model.optimize() # trigger SCIP to solve the problem

Now let's query the optimal objective value, and print primal and dual solution values using `model.getVal` for the primal solution and `model.getDualSolVal` for the dual solution (dual solutions can only be obtained for linear optimization models). 

Note that these functions can be called without arguments (to obtain the whole vector), or with a variable/constraint object to get one or more values.

In [ ]:
# SCIP - Python Interface

print(f'Objective value={model.getObjVal()}')

print(f'Solution: x={model.getVal(x)}, y={model.getVal(y)}')

print(f'Dual: constr1->{model.getDualSolVal(constr1)}, constr2->{model.getDualSolVal(constr2)}')

### Part 2S - Modeling and solving a Knapsack problem (SCIP)

Now let's move on to Mixed-Integer Programming: let's formulate and solve a [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem) with the following value/weight vectors:

$$
\begin{array}{lllrrrrrrr}
v& =& (12,&15,& 9,&11,& 8,& 7,&5)\\
w& =& (13, &18,& 9,& 12,& 8,& 10,& 4)
\end{array}
$$

and with knapsack capacity $C=40$. The formulation of a knapsack problem is as follows:

$$
\begin{array}{lllrrrrrrr}
\max & \sum_{i=1}^n v_i x_i\\
\textrm{s.t.} & \sum_{i=1}^n w_i x_i \le C\\
& x_i \in \{0,1\}, \forall i=1,\ldots{},n
\end{array}
$$

In [ ]:
# SCIP - Python Interface

v = [12,15,9,11,8, 7,5]
w = [13,18,9,12,8,10,4]
C = 40

n = len(v)

from pyscipopt import Model, quicksum

model = Model()

# it is possible to create variables inline, but usually a dictionary is used
x = {}
for i in range(n):
    x[i] = model.addVar(vtype="B") # B - binary

# add the constraint and objective function here
k_con = quicksum(w[i]*x[i] for i in range(n)) <= C
model.addCons(k_con)

k_obj = quicksum(v[i]*x[i] for i in range(n))
model.setObjective(k_obj, sense="maximize")

model.optimize()

print(model.getBestSol())